In [1]:
# Importações

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn import svm
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    cohen_kappa_score,
    roc_auc_score,
    confusion_matrix
)

import pandas as pd
import numpy as np

from nltk.corpus import stopwords

import re

import itertools

# Cross validate

from sklearn.model_selection import cross_validate

# Modelos

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [2]:
# Importação do Dataframe contendo títulos de notícias relacionadas a saúde.

dataframe = pd.read_csv('../combined_df.zip')

display(dataframe)

,title,link,date,Site,noticia_falsa,corpo_texto
0,"Ken Frazier, Merck, disse pressa vacinar liber...",https://www.boatos.org/saude/ken-frazier-merck...,30/10/2020,NaN,0,“ prometer vacinar covid gravar desserviço o o...
1,Coronavírus: Últimas notícias sabemos sexta-fe...,https://noticias.uol.com.br/saude/ultimas-noti...,08/05/2020 13h09Atualizada em 08/05/2020 22h35,NaN,1,particularidade perigoso o brasil pandemia cor...
2,menino indiano prever coronavírus antar pandemia,https://www.e-farsas.com/um-menino-indiano-pre...,24/06/2020,NaN,0,o história ganhar fama rede social e diverso s...
3,Tuíte engana afirmar sol mata coronavírus,https://projetocomprova.com.br/publicações/tui...,2020/09/10,NaN,0,tuíte referência o estimativo estudar basear m...
4,Vietnã isola cidade registrou surto covid após...,https://noticias.uol.com.br/ultimas-noticias/e...,28/07/2020 18h46,NaN,1,O Vietnã suspendeu a partir de hoje todas as r...
...,...,...,...,...,...,...
4764,Marcos Pontes voluntário testes medicamento co...,https://noticias.uol.com.br/politica/ultimas-n...,31/07/2020 11h21,NaN,1,o ministrar ciência tecnologia e inovação marc...
4765,feirar chinês vender rato morcego origem coron...,https://www.e-farsas.com/feira-chinesa-vende-r...,29/01/2020,NaN,0,o vídeo minuto duração e espalhar rapidamente ...
4766,Não evidências científicas sobre necessidade t...,https://checamos.afp.com//nao-ha-evidencias-ci...,4/06/2021 às 22:25,NaN,0,“ trocar escovar covid-19 precisar trocar esco...
4767,Coronavírus atual pode versão contagiosa origi...,https://www.uol.com.br/vivabem/noticias/afp/20...,02/07/2020 21h06Atualizada em 03/07/2020 09h02,NaN,1,o variante sars-cov-2 dominante infecto célula...


In [3]:
# Cria dataframe para processamento
df = dataframe.copy()

# Transformar a coluna título em string
df['title'] = df['title'].astype(str)

In [4]:
# Variáveis para padronização/configuração

scores = ['accuracy', 'precision', 'roc_auc']
usar_links = False

In [5]:
# Checar se há títulos vazios
non_empty_titles = df['title'].apply(lambda x: len(x.strip()) > 0)
non_empty_links = df['link'].apply(lambda x: len(x.strip()) > 0)

# Checar o resultado
print(non_empty_titles.head())
print("Há documentos vazios?", not non_empty_titles.all())
print("Há links vazios?", not non_empty_links.all())
print("Número de documentos vazios:", (~non_empty_titles).sum())
print("Número de links vazios:", (~non_empty_links).sum())

0    True
1    True
2    True
3    True
4    True
Name: title, dtype: bool
Há documentos vazios? False
Há links vazios? False
Número de documentos vazios: 0
Número de links vazios: 0


In [6]:
# Iniciar o processo de TF-IDF
tfidf_vectorizer = TfidfVectorizer()

# Nesta célula, será aplicado o TF-IDF aos títulos das notícias, caso não haja nenhum vazio
temp_tit = df['title'].tolist()
if non_empty_titles.all():
    titles_tfidf = tfidf_vectorizer.fit_transform(temp_tit)
    print('TF-IDF aplicado com sucesso.')
    tfidf = titles_tfidf
else:
    print('Há documentos vazios ou apenas stop words.')

TF-IDF aplicado com sucesso.


In [7]:
def split_and_clean(link):
# Dividir o link por '/', '-', '%', '.', e ':'
    words = re.split(r'[/\-\%\.\:\_]', link)
    # Remover palavras indesejadas, stopwords e letras isoladas, mantendo acentos
    unwanted = {'https', 'www', 'com', 'br', 'pt', 'html', 'htm', 'ghtml', 'php'}
    cleaned_words = [
    word for word in words
    if not any(unwanted_word in word.lower() for unwanted_word in unwanted)
    and len(word) > 1
    ]
    return cleaned_words

if usar_links:

    # Tratando os links
    df['processed_links'] = df['link'].apply(split_and_clean)

    # TF-IDF para os combinação título e link
    temp_link = [' '.join(i) for i in df['processed_links']]
    temp_tit_link = [' '.join(list(a)) for a in zip(temp_tit, temp_link)]
    print(temp_tit_link)
    if non_empty_links.all():
        tit_link_tfidf = tfidf_vectorizer.fit_transform(temp_tit_link)
        print('TF-IDF aplicado com sucesso.')
        tfidf = tit_link_tfidf
    else:
        print('Há items vazios ou apenas stop words.')

In [8]:
# Criação de vetores
X = tfidf
y = df['noticia_falsa']

titulos = df['title'].tolist()

print("Dimensões de X:", X.shape)
print("Dimensões de y:", y.shape)

Dimensões de X: (4769, 8505)
Dimensões de y: (4769,)


## Modelos de Classificação
### Árvore de Decisão

In [9]:
# Padronizando scores para todos os modelos.

scores = ['accuracy', 'precision', 'roc_auc']

In [10]:
# Cria modelo
modelo_ad = DecisionTreeClassifier()
modelo_ad

# Validação cruzada
resultados_ad = cross_validate(modelo_ad, X, y, scoring=scores)

acc_ad = resultados_ad['test_accuracy'].mean()
prec_ad = resultados_ad['test_precision'].mean()
rocauc_ad = resultados_ad['test_roc_auc'].mean()
print(f'Acurácia: {acc_ad:.3f}\nPrecisão: {prec_ad:.3f}\nROC-AUC: {rocauc_ad:.3f}')

Acurácia: 0.867
Precisão: 0.921
ROC-AUC: 0.855


### Floresta Aleatória

In [11]:
# Cria modelo
modelo_rf = RandomForestClassifier()
modelo_rf

# Validação cruzada
resultados_rf = cross_validate(modelo_rf, X, y, scoring=scores)

acc_rf = resultados_rf['test_accuracy'].mean()
prec_rf = resultados_rf['test_precision'].mean()
rocauc_rf = resultados_rf['test_roc_auc'].mean()
print(f'Acurácia: {acc_rf:.3f}\nPrecisão: {prec_rf:.3f}\nROC-AUC: {rocauc_rf:.3f}')

Acurácia: 0.887
Precisão: 0.910
ROC-AUC: 0.941


### Modelo Regressão Logística

In [12]:
# Cria modelo
modelo_lr = LogisticRegression()
modelo_lr

# Validação cruzada
resultados_lr = cross_validate(modelo_lr, X, y, scoring=scores)

acc_lr = resultados_lr['test_accuracy'].mean()
prec_lr = resultados_lr['test_precision'].mean()
rocauc_lr = resultados_lr['test_roc_auc'].mean()
print(f'Acurácia: {acc_lr:.3f}\nPrecisão: {prec_lr:.3f}\nROC-AUC: {rocauc_lr:.3f}')

Acurácia: 0.867
Precisão: 0.842
ROC-AUC: 0.961


### SVC

In [13]:
# Parâmetros
C = 77
kernel = 'rbf'
gamma = 0.01

# Cria modelo
modelo_svm = svm.SVC(C=C, kernel=kernel, gamma=gamma)

# Validação cruzada
resultados_svm = cross_validate(modelo_svm, X, y, scoring=scores)

acc_svm = resultados_svm['test_accuracy'].mean()
prec_svm = resultados_svm['test_precision'].mean()
rocauc_svm = resultados_svm['test_roc_auc'].mean()
print(f'Acurácia: {acc_svm:.3f}\nPrecisão: {prec_svm:.3f}\nROC-AUC: {rocauc_svm:.3f}')

Acurácia: 0.916
Precisão: 0.908
ROC-AUC: 0.963


## Predição teste com dados externos

In [14]:
df_externo = pd.read_csv('../Notícias Fake e Verdadeiras.txt', header=None, sep='\t', on_bad_lines='skip')
df_externo

,0,1,2
0,1,Vacinas contra a Covid-19 são mais perigosas q...,0
1,2,Vacina tem como objetivo matar seres humanos,0
2,3,Profissionais de saúde morrem por ataque cardí...,0
3,4,Butantan não confirma eficácia da CoronaVac em...,0
4,5,Vacinas de RNA mensageiro vão provocar morte e...,0
5,6,A vacina contra a Covid-19 vai modificar o DNA...,0
6,7,A vacina contra a Covid-19 tem chip líquido e ...,0
7,8,Imunizantes contra Covid-19 estão relacionados...,0
8,9,Vacinas contra Covid-19 criam campo magnético ...,0
9,10,CoronaVac não tem comprovação científica,0


In [15]:
stop_words = set(stopwords.words('portuguese'))

def trata_padrao(titulo, usar_links=False, link=None):
    '''
    Função para padronizar o tratamento dos títulos.

    Args:
    titulo (str) = título da notícia
    usar_links (bool) = usar ou não o link
    link (str) = link da notícia
    '''

    # Pré-processamento do titulo

    titulo = str(titulo).lower()
    titulo = nltk.word_tokenize(titulo)
    titulo = [word for word in titulo if word not in stop_words]
    titulo = [' '.join(titulo)]
    temp_tit = titulo

    # se usar link, pre processar link
    if usar_links:
        link = [split_and_clean(link)]
        temp_link = [' '.join(i) for i in link]
        print(temp_tit,temp_link)
        temp_tit_link = [' '.join(list(a)) for a in zip(temp_tit, temp_link)]
        processed = temp_tit_link
    else:
        processed = titulo

    return processed

In [16]:
# Define modelos para teste
modelos = [modelo_ad, modelo_rf, modelo_lr, modelo_svm]

# Treina modelos com todos os dados
for modelo in modelos:
    modelo.fit(X, y)

# Função para predizer novos títulos
def predict_title(titulo):
    preprocessed_title = trata_padrao(titulo)

    tfidf_vector = tfidf_vectorizer.transform(preprocessed_title)
    preds = dict()

    for modelo in modelos:
        preds[modelo] = modelo.predict(tfidf_vector)

    return preds

In [17]:
def comparar(df):
    '''
    Função para comparar predições de diferentes modelos com o target

    Args:
        df (pandas.df): dataframe com coluna '1' sendo os títulos/corpus, e '2' com o target/label
        
    Returns:
        list: Uma lista contendo 4 listas (uma para cada modelo: Decision Tree, Random Forest, Logistic Regression, SVC)
    '''
    DT, RF, LR, SVC = [], [], [], []
    for index, row in df.iterrows():
        text = row[1]
        target = row[2]
        pred = predict_title(text)
        pred['Target'] = [target]
        
        # Guardar as previsões em suas respectivas listas
        keys = [i for i in pred.keys()]
        DT.append(pred[keys[0]][0])
        RF.append(pred[keys[1]][0])
        LR.append(pred[keys[2]][0])
        SVC.append(pred[keys[3]][0])
        
        print(f'Notícia: {text}\n')
        for i in pred:
            print("\t{}\t{}".format(i, pred[i]))
        print('-----------------------------------------------------')
        print()
        
    return [DT, RF, LR, SVC]
        

# Prediz df externo    
predictions = comparar(df_externo)

Notícia: Vacinas contra a Covid-19 são mais perigosas que o próprio vírus

	DecisionTreeClassifier()	[0]
	RandomForestClassifier()	[1]
	LogisticRegression()	[0]
	SVC(C=77, gamma=0.01)	[1]
	Target	[0]
-----------------------------------------------------

Notícia: Vacina tem como objetivo matar seres humanos

	DecisionTreeClassifier()	[1]
	RandomForestClassifier()	[0]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[0]
	Target	[0]
-----------------------------------------------------

Notícia: Profissionais de saúde morrem por ataque cardíaco em decorrência da vacina

	DecisionTreeClassifier()	[1]
	RandomForestClassifier()	[1]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[0]
-----------------------------------------------------

Notícia: Butantan não confirma eficácia da CoronaVac em idosos

	DecisionTreeClassifier()	[1]
	RandomForestClassifier()	[1]
	LogisticRegression()	[1]
	SVC(C=77, gamma=0.01)	[1]
	Target	[0]
-----------------------------------------------------

N

In [18]:
# Calcula score de cada método para df externo
target = df_externo[2].tolist()

acc_dt_e = accuracy_score(target, predictions[0])
acc_rf_e = accuracy_score(target, predictions[1])
acc_lr_e = accuracy_score(target, predictions[2])
acc_svc_e = accuracy_score(target, predictions[3])

prec_dt_e = precision_score(target, predictions[0])
prec_rf_e = precision_score(target, predictions[1])
prec_lr_e = precision_score(target, predictions[2])
prec_svc_e = precision_score(target, predictions[3])

print('---- Decision Tree ----')
print(f'Acurácia: {acc_dt_e:.3f}\nPrecisão: {prec_dt_e:.3f}')
print()
print('---- Random Forest ----')
print(f'Acurácia: {acc_rf_e:.3f}\nPrecisão: {prec_rf_e:.3f}')
print()
print('---- Linear Regression ----')
print(f'Acurácia: {acc_lr_e:.3f}\nPrecisão: {prec_lr_e:.3f}')
print()
print('---- SVC ----')
print(f'Acurácia: {acc_svc_e:.3f}\nPrecisão: {prec_svc_e:.3f}')
print()

---- Decision Tree ----
Acurácia: 0.650
Precisão: 0.600

---- Random Forest ----
Acurácia: 0.550
Precisão: 0.529

---- Linear Regression ----
Acurácia: 0.600
Precisão: 0.562

---- SVC ----
Acurácia: 0.650
Precisão: 0.600

